In [13]:
# Import library utama
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [14]:
url = "https://books.toscrape.com/"
response = requests.get(url)
print(response)   # <Response [200]> artinya sukses


<Response [200]>


In [15]:
pages = []
i = 1

while True:
    url_page = f"{url}catalogue/page-{i}.html"
    page_response = requests.get(url_page)

    if page_response.status_code == 200:
        pages.append(url_page)
        print(f"Response {page_response.status_code} | Halaman: {i}")
        i += 1
    else:
        break  # berhenti jika halaman tidak ditemukan

print(f"\nTotal halaman ditemukan: {len(pages)}")


Response 200 | Halaman: 1
Response 200 | Halaman: 2
Response 200 | Halaman: 3
Response 200 | Halaman: 4
Response 200 | Halaman: 5
Response 200 | Halaman: 6
Response 200 | Halaman: 7
Response 200 | Halaman: 8
Response 200 | Halaman: 9
Response 200 | Halaman: 10
Response 200 | Halaman: 11
Response 200 | Halaman: 12
Response 200 | Halaman: 13
Response 200 | Halaman: 14
Response 200 | Halaman: 15
Response 200 | Halaman: 16
Response 200 | Halaman: 17
Response 200 | Halaman: 18
Response 200 | Halaman: 19
Response 200 | Halaman: 20
Response 200 | Halaman: 21
Response 200 | Halaman: 22
Response 200 | Halaman: 23
Response 200 | Halaman: 24
Response 200 | Halaman: 25
Response 200 | Halaman: 26
Response 200 | Halaman: 27
Response 200 | Halaman: 28
Response 200 | Halaman: 29
Response 200 | Halaman: 30
Response 200 | Halaman: 31
Response 200 | Halaman: 32
Response 200 | Halaman: 33
Response 200 | Halaman: 34
Response 200 | Halaman: 35
Response 200 | Halaman: 36
Response 200 | Halaman: 37
Response 2

In [16]:
book_urls = []

for page in pages:
    page_response = requests.get(page)
    page_soup = BeautifulSoup(page_response.content, "html.parser")

    # Ambil semua link buku di halaman tsb
    links = [
        "https://books.toscrape.com/catalogue/" + link.get("href").replace("../../../", "")
        for link in page_soup.select("h3 a")
    ]

    book_urls.extend(links)
    print(f"{len(links)} buku ditemukan di {page}")

print(f"\nTotal URL buku terkumpul: {len(book_urls)}")


20 buku ditemukan di https://books.toscrape.com/catalogue/page-1.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-2.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-3.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-4.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-5.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-6.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-7.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-8.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-9.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-10.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-11.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-12.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-13.html
20 buku ditemukan di https://books.toscrape.com/catalogue/page-14.html
20 buku ditemuk

In [17]:
books_data = []

for url_book in book_urls:
    response = requests.get(url_book)
    soup = BeautifulSoup(response.text, "html.parser")

    info = soup.select("table.table.table-striped tr")

    data = {
        "category": soup.select_one("ul.breadcrumb li:nth-child(3) a").text.strip(),
        "code": info[0].select_one("td").text.strip(),
        "cover": "https://books.toscrape.com/catalogue/" + soup.select_one("div.item.active img")["src"].replace("../../", ""),
        "title": soup.select_one("div.product_main h1").text.strip(),
        "rating": soup.select_one("p.star-rating")["class"][1],
        "price (excl. tax)": info[2].select_one("td").text.strip(),
        "price (incl. tax)": info[3].select_one("td").text.strip(),
        "tax": info[4].select_one("td").text.strip(),
        "stock status": soup.select_one("p.availability").text.strip(),
        "number of stock available": ''.join(filter(str.isdigit, soup.select_one("p.availability").text)),
        "description": soup.select_one("article.product_page > p").text.strip() if soup.select_one("article.product_page > p") else "",
        "number of reviews": info[6].select_one("td").text.strip(),
    }

    books_data.append(data)
    print(f"Scraped: {data['title']}")
    time.sleep(0.2)  # jeda agar tidak membebani server


Scraped: A Light in the Attic
Scraped: Tipping the Velvet
Scraped: Soumission
Scraped: Sharp Objects
Scraped: Sapiens: A Brief History of Humankind
Scraped: The Requiem Red
Scraped: The Dirty Little Secrets of Getting Your Dream Job
Scraped: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
Scraped: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
Scraped: The Black Maria
Scraped: Starving Hearts (Triangular Trade Trilogy, #1)
Scraped: Shakespeare's Sonnets
Scraped: Set Me Free
Scraped: Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
Scraped: Rip it Up and Start Again
Scraped: Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
Scraped: Olio
Scraped: Mesaerion: The Best Science Fiction Stories 1800-1849
Scraped: Libertarianism for Beginners
Scraped: It's Only the Himalayas
Scraped: In Her Wake
Scraped: How Music Works
Scraped: Foolproof Preserving: A Guide to Small

In [26]:
import pandas as pd
from tabulate import tabulate

# Buat DataFrame
df_books = pd.DataFrame(books_data)

# Simpan ke CSV
df_books.to_csv("books_scraped.csv", index=False, encoding="utf-8-sig")

# Tampilkan hasil tabel
print(f"\n Hasil Scraping: {len(df_books)} Buku Ditemukan\n")
print(tabulate(df_books.head(), headers='keys', tablefmt='grid', showindex=False))



 Hasil Scraping: 1000 Buku Ditemukan

+--------------------+------------------+---------------------------------------------------------------------------------------------+---------------------------------------+----------+---------------------+---------------------+--------+-------------------------+-----------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------